In [388]:
# helper functions

# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):    
    sys.stdout.write('.')
    sys.stdout.flush()
    time.sleep(t)
    

def f(x):
    if x >= 10:
        return 10
    elif x <= 0:
        return 1
       
    else:
        return x 
    

def g(x):
    if x >= 5:
        return 10
    elif x <= 0:
        return 1
    else:
        return 5 
    
    
def h(x):
    return x['pt_rankStars'] + x['pt_rankStatewidePercentage'] + x['pt_averageStandardScore'] + x['pt_rankMovement'] + x['pt_pupilTeacherRatio']


def i(x):
    return (x['high_score'] + x['middle_score'] + x['elementary_score']) / 15


def k(x):
    if x >= 100:
        return 10
    elif x == 100:
        return 7.5
    elif x >= 50 and  x < 100:
        return 5
    elif x > 0 and  x < 50:
        return 1
    else:
        return 0


def l(x):
    return (x['score_grocery_stores'] + x['score_police_stations'] + x['score_transit_stations'] + x['score_museums']) / 4

In [389]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [196]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
required_zips = zipcodes.ziplist

Target city: atlanta
Search radius: 5


In [199]:
# get lat, lng coords from ziplist
url = "https://maps.googleapis.com/maps/api/geocode/json?"
response = []
latlngs = []

for z in required_zips:
    params = {
        "address":z,
        "key":gkey
    }

    res = requests.get(url, params=params).json()
    response.append({"response":res, "zipcode": z})

for r in response:
    try:
        zc = r['zipcode']
        lat = r['response']['results'][0]['geometry']['location']['lat']
        lng = r['response']['results'][0]['geometry']['location']['lng']
        
        latlngs.append({
            "lat":lat,
            "lng":lng,
            "zipcode":zc
        })

    except:
        continue

# Neighbourhood Analysis

In [203]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
ttypes = ['grocery_or_supermarket', 'police', 'transit_station', 'museum']

zipcodes = []
target_coordinates = []

museums = []
grocery_stores = []
police_stations = []
transit_stations = []

for l in latlngs:
    curzip = l['zipcode']
    curlatlng = f"{l['lat']},{l['lng']}"
    
    for i,t in enumerate(ttypes):
        curtype = t
        print(f"zipcode: {curzip}, latlng: {curlatlng}, target_type: {curtype}")
        
        params = {
            "target_search": curtype,
            "location": curlatlng,
            "radius": 1609, # TODO: pull this in for dynamic search radius
            "type": curtype,
            "key":gkey
        }

        response = requests.get(base_url, params=params).json()

        # populate list items
        for item in response["results"]:
            if(i == 0):
                grocery_stores.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            elif(i == 1):
                police_stations.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            elif(i == 2):
                transit_stations.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })
            else:
                museums.append({
                    "zipcode":curzip,
                    "loc":curlatlng,
                    "item":item["name"],
                    "type":curtype
                })

zipcode: 30330, latlng: 33.9542004,-84.30459019999999, target_type: grocery_or_supermarket
zipcode: 30330, latlng: 33.9542004,-84.30459019999999, target_type: police
zipcode: 30330, latlng: 33.9542004,-84.30459019999999, target_type: transit_station
zipcode: 30330, latlng: 33.9542004,-84.30459019999999, target_type: museum
zipcode: 30315, latlng: 33.7026678,-84.38574419999999, target_type: grocery_or_supermarket
zipcode: 30315, latlng: 33.7026678,-84.38574419999999, target_type: police
zipcode: 30315, latlng: 33.7026678,-84.38574419999999, target_type: transit_station
zipcode: 30315, latlng: 33.7026678,-84.38574419999999, target_type: museum
zipcode: 30316, latlng: 33.723039,-84.3436881, target_type: grocery_or_supermarket
zipcode: 30316, latlng: 33.723039,-84.3436881, target_type: police
zipcode: 30316, latlng: 33.723039,-84.3436881, target_type: transit_station
zipcode: 30316, latlng: 33.723039,-84.3436881, target_type: museum
zipcode: 30310, latlng: 33.7321902,-84.42251859999999, ta

zipcode: 30361, latlng: 33.7877341,-84.38081729999999, target_type: grocery_or_supermarket
zipcode: 30361, latlng: 33.7877341,-84.38081729999999, target_type: police
zipcode: 30361, latlng: 33.7877341,-84.38081729999999, target_type: transit_station
zipcode: 30361, latlng: 33.7877341,-84.38081729999999, target_type: museum
zipcode: 30306, latlng: 33.7845409,-84.3489467, target_type: grocery_or_supermarket
zipcode: 30306, latlng: 33.7845409,-84.3489467, target_type: police
zipcode: 30306, latlng: 33.7845409,-84.3489467, target_type: transit_station
zipcode: 30306, latlng: 33.7845409,-84.3489467, target_type: museum
zipcode: 30325, latlng: 33.8373615,-84.4201604, target_type: grocery_or_supermarket
zipcode: 30325, latlng: 33.8373615,-84.4201604, target_type: police
zipcode: 30325, latlng: 33.8373615,-84.4201604, target_type: transit_station
zipcode: 30325, latlng: 33.8373615,-84.4201604, target_type: museum
zipcode: 31191, latlng: 37.8152808,-119.9390509, target_type: grocery_or_supermar

In [390]:
# generate dataframes
gdf = pd.DataFrame(grocery_stores)
tdf = pd.DataFrame(transit_stations)
pdf = pd.DataFrame(police_stations)
mdf = pd.DataFrame(museums)

In [391]:
# group data by zipcode
grocery_count = gdf[['item', 'zipcode']].groupby('zipcode').count()
transit_count = tdf[['item', 'zipcode']].groupby('zipcode').count()
police_count = pdf[['item', 'zipcode']].groupby('zipcode').count()
museum_count = mdf[['item', 'zipcode']].groupby('zipcode').count()

In [392]:
# merge dataframes
gt_count = grocery_count.merge(transit_count, on='zipcode')
gt_count = gt_count.rename(columns={
    "item_x":"num_grocery_stores",
    "item_y":"num_transit_stations"
})

gtp_count = gt_count.merge(police_count, on='zipcode')
gtp_count = gtp_count.rename(columns={
    "item":"num_police_stations"
})

gtpm_count = gtp_count.merge(museum_count, on="zipcode")
gtpm_count = gtpm_count.rename(columns={
    "item":"num_museums"
})

In [393]:
# extract grocery stores, police stations, museums, transit stations and assign scores
average_grocery = gtpm_count["num_grocery_stores"].mean()
average_police = gtpm_count["num_police_stations"].mean()
average_museum = gtpm_count["num_museums"].mean()
average_transit_station = gtpm_count["num_transit_stations"].mean()

# assign scores
gtpm_count["avg_grocery_stores"] = (gtpm_count["num_grocery_stores"]/average_grocery*100).fillna(0.0).astype(int)
gtpm_count["avg_police_stations"] = (gtpm_count["num_police_stations"]/average_police*100).fillna(0.0).astype(int)
gtpm_count["avg_transit_stations"] = (gtpm_count["num_transit_stations"]/average_transit_station*100).fillna(0.0).astype(int)
gtpm_count["avg_museums"] = (gtpm_count["num_museums"]/average_museum*100 ).fillna(0.0).astype(int)
gtpm_count

,num_grocery_stores,num_transit_stations,num_police_stations,num_museums,avg_grocery_stores,avg_police_stations,avg_transit_stations,avg_museums
zipcode,,,,,,,,
30302,3,20,17,13,120,176,100,110
30303,1,20,18,20,40,187,100,170
30308,4,20,2,2,160,20,100,17
30312,2,20,15,12,80,156,100,102
30313,1,20,12,12,40,124,100,102
30314,3,20,2,5,120,20,100,42
30317,1,20,1,2,40,10,100,17
30332,4,20,4,9,160,41,100,76
30334,3,20,16,13,120,166,100,110


In [394]:
gtpm_count["score_grocery_stores"] = gtpm_count.apply(lambda row: k(row['avg_grocery_stores']), axis=1)
gtpm_count["score_police_stations"] = gtpm_count.apply(lambda row: k(row['avg_police_stations']), axis=1)
gtpm_count["score_transit_stations"] = gtpm_count.apply(lambda row: k(row['avg_transit_stations']), axis=1)
gtpm_count["score_museums"] = gtpm_count.apply(lambda row: k(row['avg_museums']), axis=1)
gtpm_count["total_score"] = gtpm_count.apply(lambda row: l(row), axis=1)
gtpm_count = gtpm_count.drop(columns=['num_grocery_stores','num_police_stations','num_transit_stations','num_museums'])
gtpm_count

,avg_grocery_stores,avg_police_stations,avg_transit_stations,avg_museums,score_grocery_stores,score_police_stations,score_transit_stations,score_museums,total_score
zipcode,,,,,,,,,
30302,120,176,100,110,10,10,10,10,10.00
30303,40,187,100,170,1,10,10,10,7.75
30308,160,20,100,17,10,1,10,1,5.50
30312,80,156,100,102,5,10,10,10,8.75
30313,40,124,100,102,1,10,10,10,7.75
30314,120,20,100,42,10,1,10,1,5.50
30317,40,10,100,17,1,1,10,1,3.25
30332,160,41,100,76,10,1,10,5,6.50
30334,120,166,100,110,10,10,10,10,10.00


# Price Analysis

In [208]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/assessment/detail?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', 'CONDOMINIUM', 
              'APARTMENT', 'CABIN', 'DUPLEX', 
              'MANUFACTURED HOME', 'MOBILE HOME',
              'TRIPLEX', 'TOWNHOUSE/ROWHOUSE']

sys.stdout.write('working..')

for z in latlngs:
    curzip = z['zipcode']
    
    for pt in prop_types:
        curpt = pt

        params = {
            "postalcode": curzip,
            "propertytype": curpt
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(8)

working........................................................................................................................................................................................................................................................................................................................................................

In [400]:
prop_res = []
for p in property_results:
    try:
        for q in p['property']:
            zipcode = q['address']['postal1']
            mkt_val = q['assessment']['market']['mktttlvalue']
            prop_res.append({"zipcode":zipcode, "mkt_val":mkt_val})
    except Exception as e:
        print(e)

In [405]:
prop_df = pd.DataFrame(prop_res)
avg_mkt_val = round(prop_df.groupby(['zipcode', 'mkt_val']).sum() / len(prop_res), 2)

In [409]:
gtpm_count.merge(avg_mkt_val, on="zipcode") \
    .rename(columns={"mkt_val":"avg_mkt_val"}) \
    .drop(columns=['avg_grocery_stores','avg_police_stations','avg_transit_stations','avg_museums']) \
    .style.format({"avg_mkt_val":"${:,.2f}"})

ValueError: style is not supported for non-unique indicies.

# School Analysis

In [29]:
# Inititalize arrays for all the various parameters returned from response of call that is needed
postal = []
schoolLevel = []
rankStars = []
rankStatewidePercentage = []
averageStandardScore = []
rankMovement = []
pupilTeacherRatio = []

In [30]:
school_base_url="https://api.schooldigger.com/v1.1/schools?"

for zip in required_zips:
    params = {
        "appID":'cdcbd9a2',
        "appKey":'f21463a388bc041e25afa5dc32fb5014',
        "st":'GA',
        "zip":zip
    }

    # assemble url and make API request
    response = requests.get(school_base_url, params=params).json()
    response_df = pd.DataFrame(response)
    
    numberofresults = len(response_df)
    for i in range(numberofresults):
        schoolLevel.append(response_df["schoolList"][i].get("schoolLevel",np.nan))
        rankMovement.append(response_df["schoolList"][i].get("rankMovement",np.nan))
        postal.append(response_df["schoolList"][i]["address"].get("zip",np.nan))            

        try:
            rankStars.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStars",np.nan))
            rankStatewidePercentage.append(response_df["schoolList"][i]["rankHistory"][0].get("rankStatewidePercentage",np.nan))
            averageStandardScore.append(response_df["schoolList"][i]["rankHistory"][0].get("averageStandardScore",np.nan))

        except (KeyError, IndexError, TypeError):
            rankStars.append(np.nan)
            rankStatewidePercentage.append(np.nan)
            averageStandardScore.append(np.nan)
                
        try:    
            pupilTeacherRatio.append(response_df["schoolList"][i]["schoolYearlyDetails"][0].get("pupilTeacherRatio",np.nan))
    
        except (KeyError, IndexError, TypeError):
            pupilTeacherRatio.append(np.nan)

        time.sleep(30)

In [31]:
# create a data frame from all schools
school_dict = {
        "postal":postal,
        "schoolLevel":schoolLevel,
        "rankStars":rankStars,
        "rankStatewidePercentage":rankStatewidePercentage,
        "averageStandardScore":averageStandardScore,
        "rankMovement":rankMovement,
        "pupilTeacherRatio":pupilTeacherRatio
}

school_dict_pd = pd.DataFrame(school_dict)
school_dict_pd.replace(np.nan, 0, inplace=True)
school_dict_pd

,postal,schoolLevel,rankStars,rankStatewidePercentage,averageStandardScore,rankMovement,pupilTeacherRatio
0,99999,Elementary,1.0,7.41,89.49,29.0,12.6
1,99999,Middle,1.0,50.58,59.08,64.0,15.4
2,99999,Elementary,3.0,87.66,51.11,-31.0,13.5
3,99999,Private,0.0,0.00,0.00,0.0,3.8
4,99999,Elementary,1.0,95.10,79.08,-75.0,13.1
5,99999,Elementary,3.0,14.24,20.22,-1.0,12.0
6,99999,High,0.0,0.00,0.00,0.0,0.0
7,99999,High,1.0,94.13,65.74,57.0,23.3


In [32]:
# seperate middle, elementary and high and assign scores
school_elementary_pd = school_dict_pd[school_dict_pd['schoolLevel'] == 'Elementary']
school_elementary_pd = school_elementary_pd.copy()
averagerankStars = school_elementary_pd["rankStars"].mean()

avergaerankStatewidePercentage = school_elementary_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_elementary_pd["averageStandardScore"].mean()
avergaerankMovement = school_elementary_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_elementary_pd["pupilTeacherRatio"].mean()

# Assign scores
school_elementary_pd["pg_avg_rankStars"] = (school_elementary_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankStatewidePercentage"] = (school_elementary_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_averageStandardScore"] = (school_elementary_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_elementary_pd["pg_avg_rankMovement"] = school_elementary_pd["rankMovement"]/avergaerankMovement *10
school_elementary_pd["pg_avg_pupilTeacherRatio"] = (school_elementary_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_elementary_pd['pt_rankStars'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_elementary_pd['pt_rankStatewidePercentage'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_elementary_pd['pt_averageStandardScore'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_elementary_pd['pt_rankMovement'] = school_elementary_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_elementary_pd['pt_pupilTeacherRatio'] = school_elementary_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_elementary_pd['total_points'] = school_elementary_pd.apply(lambda row: h(row),axis=1)
school_elementary_points = school_elementary_pd[['postal','total_points']]
elementary_groupby = school_elementary_points.groupby('postal').mean()

# elementary_groupby
school_elementary_pd

,postal,schoolLevel,rankStars,rankStatewidePercentage,averageStandardScore,rankMovement,pupilTeacherRatio,pg_avg_rankStars,pg_avg_rankStatewidePercentage,pg_avg_averageStandardScore,pg_avg_rankMovement,pg_avg_pupilTeacherRatio,pt_rankStars,pt_rankStatewidePercentage,pt_averageStandardScore,pt_rankMovement,pt_pupilTeacherRatio,total_points
0,99999,Elementary,1.0,7.41,89.49,29.0,12.6,5,1,14,-14.871795,9,5,1,10,10,9,35
2,99999,Elementary,3.0,87.66,51.11,-31.0,13.5,15,17,8,15.897436,10,10,10,8,1,10,39
4,99999,Elementary,1.0,95.10,79.08,-75.0,13.1,5,18,13,38.461538,10,5,10,10,1,10,36
5,99999,Elementary,3.0,14.24,20.22,-1.0,12.0,15,2,3,0.512821,9,10,2,3,1,9,25


In [33]:
# seperate middle, middle and high and assign scores
school_middle_pd = school_dict_pd[school_dict_pd['schoolLevel']=='Middle']
school_middle_pd = school_middle_pd.copy()
averagerankStars = school_middle_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_middle_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_middle_pd["averageStandardScore"].mean()
avergaerankMovement = school_middle_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_middle_pd["pupilTeacherRatio"].mean()

# Assign scores
school_middle_pd["pg_avg_rankStars"] = (school_middle_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankStatewidePercentage"] = (school_middle_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_averageStandardScore"] = (school_middle_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_middle_pd["pg_avg_rankMovement"] = school_middle_pd["rankMovement"]/avergaerankMovement *10
school_middle_pd["pg_avg_pupilTeacherRatio"] = (school_middle_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_middle_pd['pt_rankStars'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_middle_pd['pt_rankStatewidePercentage'] = school_middle_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_middle_pd['pt_averageStandardScore'] = school_middle_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_middle_pd['pt_rankMovement'] = school_middle_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_middle_pd['pt_pupilTeacherRatio'] = school_middle_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_middle_pd['total_points'] = school_middle_pd.apply(lambda row: h(row),axis=1)

school_middle_points=school_middle_pd[['postal','total_points']]
middle_groupby=school_middle_points.groupby('postal').mean()
middle_groupby

,total_points
postal,
99999,50


In [34]:
# seperate high, high and high and assign scores
school_high_pd = school_dict_pd[school_dict_pd['schoolLevel']=='High']
school_high_pd = school_high_pd.copy()
averagerankStars = school_high_pd["rankStars"].mean()
avergaerankStatewidePercentage = school_high_pd["rankStatewidePercentage"].mean()
avergaeaverageStandardScore = school_high_pd["averageStandardScore"].mean()
avergaerankMovement = school_high_pd["rankMovement"].mean()
avergaepupilTeacherRatio = school_high_pd["pupilTeacherRatio"].mean()

# Assign scores
school_high_pd["pg_avg_rankStars"] = (school_high_pd["rankStars"]/averagerankStars *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankStatewidePercentage"] = (school_high_pd["rankStatewidePercentage"]/avergaerankStatewidePercentage *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_averageStandardScore"] = (school_high_pd["averageStandardScore"]/avergaeaverageStandardScore *10).fillna(0.0).astype(int)
school_high_pd["pg_avg_rankMovement"] = school_high_pd["rankMovement"]/avergaerankMovement *10
school_high_pd["pg_avg_pupilTeacherRatio"] = (school_high_pd["pupilTeacherRatio"]/avergaepupilTeacherRatio *10).fillna(0.0).astype(int)

school_high_pd['pt_rankStars'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStars']), axis=1)
school_high_pd['pt_rankStatewidePercentage'] = school_high_pd.apply(lambda row: f(row['pg_avg_rankStatewidePercentage']), axis=1)
school_high_pd['pt_averageStandardScore'] = school_high_pd.apply(lambda row: f(row['pg_avg_averageStandardScore']), axis=1)
school_high_pd['pt_rankMovement'] = school_high_pd.apply(lambda row: g(row['rankMovement']), axis=1)
school_high_pd['pt_pupilTeacherRatio'] = school_high_pd.apply(lambda row: f(row['pg_avg_pupilTeacherRatio']), axis=1)
school_high_pd['total_points'] = school_high_pd.apply(lambda row: h(row),axis=1)

school_high_points = school_high_pd[['postal','total_points']]
high_groupby = school_high_points.groupby('postal').mean()
high_groupby

,total_points
postal,
99999,27.5


In [86]:
zip_final_school_score = []

combined_df = pd.merge(high_groupby, middle_groupby, how='outer', on='postal')

zip_final_school_score = pd.merge(combined_df, elementary_groupby, how='outer', on='postal')
zip_final_school_score.replace(np.nan, 0, inplace=True)
zip_final_school_score.rename(columns={'total_points_x': 'high_score','total_points_y': 'middle_score', 'total_points': 'elementary_score'}, inplace=True)
zip_final_school_score['final_score'] = zip_final_school_score.apply(lambda row: i(row), axis=1)
zip_final_school_score

,high_score,middle_score,elementary_score,final_score
postal,,,,
99999,27.5,50,33.75,7.416667


In [ ]:
# # get properties for each zip code (10 queries/min)
# url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
# property_results = []
# prop_types = ['RESIDENTIAL (NEC)', #, 'CONDOMINIUM', 
#               'APARTMENT'] #, 'CABIN', 'DUPLEX', 
#               #'MANUFACTURED HOME', 'MOBILE HOME',
#               #'TRIPLEX', 'TOWNHOUSE/ROWHOUSE']

# sys.stdout.write('working..')

# for z in latlngs:
#     for pt in prop_types:
#         params = {
#             "postalcode": z['zipcode'],
#             "propertytype": pt,
#             "pagesize": "100"
#         }

#         res = fetch(url, params)

#         property_results.append(res)
#         wait(8)

In [ ]:
# # get attomIds from property results
# properties = []
# attomIds = []

# for p in property_results:
#     for q in p['property']:
#         properties.append(q)

# props_df = pd.DataFrame(properties)
# attomId_series = props_df['identifier']

# for a in attomId_series:
#     attomIds.append(a['attomId'])

In [ ]:
# # get home values for each property (10 queries/min)
# url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
# sys.stdout.write("working..")
# hv_results = []

# for a in attomIds:
#     params = {
#         "attomId": a,
#         "page":"1",
#         "pagesize":"100"
#     }

#     idres = fetch(url, params)
#     hv_results.append(idres)    
#     wait(10)

In [ ]:
# prop_res = pd.DataFrame(hv_results)
# props = prop_res['property']
# zcmv = []

# for i in range(len(props)):
#     zcmv.append({
#         "zipcode":props[i][0]['address']['postal1'], 
#         "avg_mkt_val":int(props[i][0]['assessment']['market']['mktTtlValue'])
#     })

# zips_mktvals_df = pd.DataFrame(zcmv).set_index('zipcode')
# mkt_avg = zips_mktvals_df.groupby('zipcode').sum()/len(props)

# mkt_avg_df = pd.DataFrame(mkt_avg)
# mkt_avg_df = mkt_avg_df.reset_index()
# mkt_avg_df.style.format({'avg_mkt_val':'${:,.2f}'})